You'll be using the Dark Sky Forecast API from Forecast.io, available at https://developer.forecast.io. It's a pretty simple API, but be sure to read the documentation!

A forecast request returns the current forecast (for the next week): 
https://api.forecast.io/forecast/APIKEY/LATITUDE,LONGITUDE

A time-machine request returns the observed weather at a given time (for many places, up to 60 years in the past): https://api.forecast.io/forecast/APIKEY/LATITUDE,LONGITUDE,TIME


1) Make a request from the Forecast.io API for where you were born (or lived, or want to visit!).

In [2]:
import requests

In [35]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/47.4079,9.4647")
weather_data = response.json()
weather_data.keys()

dict_keys(['longitude', 'currently', 'hourly', 'daily', 'latitude', 'flags', 'timezone', 'offset'])

In [36]:
print(weather_data['timezone'])

Europe/Zurich


The time zone of Trogen is correct! This is where I live.

In [37]:
print("Longitude:", weather_data['longitude'], "Latitude", weather_data['latitude'])

Longitude: 9.4647 Latitude 47.4079


The longitude is mentioned first, and then the latitude. Usually, it is the other way round.

2) What's the current wind speed? How much warmer does it feel than it actually is?

In [38]:
type(weather_data['currently'])

dict

In [39]:
weather_data['currently'].keys()

dict_keys(['precipIntensity', 'time', 'ozone', 'visibility', 'humidity', 'apparentTemperature', 'dewPoint', 'pressure', 'windSpeed', 'cloudCover', 'temperature', 'windBearing', 'precipProbability', 'summary', 'icon'])

In [40]:
print("The wind in Trogen, Switzerland, is currently blowing at", weather_data['currently']['windSpeed'], "mph.")

The wind in Trogen, Switzerland, is currently blowing at 3.97 mph.


In [41]:
weather = weather_data['currently']
Temperature = int(weather['apparentTemperature']) - int(weather['temperature'])
Celsius = round((int(weather['temperature']) - 32) * 5 / 9)
if Temperature == 0:
    print("The temperature feels exactly as you would expect it to feel, namely", weather['temperature'], "degrees Fahrenheit, or", Celsius, "degrees Celsius.")
elif Temperature > 0:
    print("It feels", Temperature, "degrees Fahrenheit warmer than the actual temperature, which is", weather['temperature'], "degrees Fahrenheit, or", Celsius, "degrees Celsius.")
else:
    print("It feels", Temperature, "degrees Fahrenheit colder than the actual temperature, which is", weather['temperature'], "degrees Fahrenheit, or", Celsius, "degrees Celsius.")

The temperature feels exactly as you would expect it to feel, namely 68.67 degrees Fahrenheit, or 20 degrees Celsius.


3) The first daily forecast is the forecast for today. For the place you decided on up above, how much of the moon is currently visible?

In [42]:
type(weather_data['daily'])

dict

In [43]:
weather_data['daily'].keys()

dict_keys(['summary', 'icon', 'data'])

In [44]:
type(weather_data['daily']['data'])

list

In [45]:
for phase in weather_data['daily']['data']:
    moon_forecast_tomorrow = phase['moonPhase']
    break
#print(phase['moonPhase'])
if moon_forecast_tomorrow == 0:
    print("Tomorrow is New Moon.")
elif moon_forecast_tomorrow > .75:
    print("Tomorrow the Moon is in a Waning Crescent phase.")
elif moon_forecast_tomorrow == .75:
    print("Tomorrow is last quarter Moon.")
elif moon_forecast_tomorrow > .5:
    print("Tommorrow the Moon is in a Waning Gibbous phase.")
elif moon_forecast_tomorrow == .5:
    print("Tommorrow is Full Moon.")
elif moon_forecast_tomorrow > .25:
    print("Tommorrow the Moon is a Waxing Gibbous phase.")
elif moon_forecast_tomorrow == .25:
    print("Tommorrow is first Quarter Moon.")
elif moon_forecast_tomorrow > 0:
    print("Tommorrow the Moon is in a Waxing Crescent phase. This is the first phase after New Moon.")


Tommorrow the Moon is in a Waxing Crescent phase. This is the first phase after New Moon.


4) What's the difference between the high and low temperatures for today?

In [46]:
print(weather_data['currently'])

{'precipIntensity': 0, 'time': 1465569903, 'ozone': 327.37, 'visibility': 6.21, 'humidity': 0.53, 'apparentTemperature': 68.67, 'dewPoint': 50.91, 'pressure': 1010.77, 'windSpeed': 3.97, 'cloudCover': 0.12, 'temperature': 68.67, 'windBearing': 10, 'precipProbability': 0, 'summary': 'Clear', 'icon': 'clear-day'}


In [47]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/47.4079,9.4647,1465420000")
hist_weather_data = response.json()

hist_weather_data.keys()




dict_keys(['longitude', 'currently', 'hourly', 'daily', 'latitude', 'flags', 'timezone', 'offset'])

In [48]:
hist_weather_data['daily'].keys()

dict_keys(['data'])

In [49]:
print(hist_weather_data['daily']['data'])

[{'moonPhase': 0.12, 'time': 1465336800, 'precipIntensityMaxTime': 1465408800, 'visibility': 6.14, 'apparentTemperatureMaxTime': 1465390800, 'humidity': 0.75, 'ozone': 344.35, 'pressure': 1019.61, 'apparentTemperatureMax': 66.79, 'temperatureMax': 66.79, 'precipType': 'rain', 'windBearing': 251, 'precipIntensityMax': 0.0519, 'dewPoint': 53.99, 'summary': 'Light rain throughout the day.', 'precipIntensity': 0.0168, 'icon': 'rain', 'temperatureMin': 57.88, 'sunriseTime': 1465356438, 'temperatureMinTime': 1465419600, 'windSpeed': 2.86, 'apparentTemperatureMinTime': 1465419600, 'temperatureMaxTime': 1465390800, 'cloudCover': 0.41, 'apparentTemperatureMin': 57.88, 'sunsetTime': 1465413482, 'precipProbability': 0.65}]


In [50]:
for n in hist_weather_data['daily']['data']:
    Min = n['temperatureMin']
    Max = n['temperatureMax']

In [51]:
Min_Max = Max - Min
Min_Max_Celsius = ((Max - 32) * 5 / 9) - ((Min - 32) * 5 / 9)
print("The diffrence between the high and low temperatures on Wedesday, June 8, in Trogen, Switzerland, was", round(Min_Max), "Fahrenheit", "or", round(Min_Max_Celsius), "Celsius.")

The diffrence between the high and low temperatures on Wedesday, June 8, in Trogen, Switzerland, was 9 Fahrenheit or 5 Celsius.


5) Loop through the daily forecast, printing out the next week's worth of predictions. I'd like to know the high temperature for each day, and whether it's hot, warm, or cold, based on what temperatures you think are hot, warm or cold.

In [52]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.7128,-74.0059")
weather_data = response.json()
weather_data.keys()

dict_keys(['longitude', 'currently', 'daily', 'hourly', 'minutely', 'latitude', 'flags', 'timezone', 'offset'])

In [53]:
weather_data['daily'].keys()

dict_keys(['summary', 'icon', 'data'])

In [60]:
print(weather_data['daily']['data'][0])

{'moonPhase': 0.2, 'time': 1465531200, 'ozone': 377.46, 'visibility': 10, 'apparentTemperatureMaxTime': 1465592400, 'humidity': 0.39, 'pressure': 1015.38, 'apparentTemperatureMax': 77.53, 'temperatureMax': 77.53, 'windBearing': 298, 'precipIntensityMax': 0, 'dewPoint': 40.1, 'summary': 'Clear throughout the day.', 'precipIntensity': 0, 'icon': 'clear-day', 'temperatureMin': 56.26, 'sunriseTime': 1465550751, 'temperatureMinTime': 1465552800, 'windSpeed': 7.19, 'apparentTemperatureMinTime': 1465552800, 'temperatureMaxTime': 1465592400, 'cloudCover': 0.13, 'apparentTemperatureMin': 56.26, 'sunsetTime': 1465604885, 'precipProbability': 0}


In [61]:
Forecast = weather_data['daily']['data']
for max_temp in Forecast:
    Convert_Celsius = (max_temp['temperatureMax'] - 32) * 5 / 9
    if Convert_Celsius > 30:
        Temperature = "hot"
    elif Convert_Celsius > 20:
        Temperature = "warm"
    else:
        Temperature = "cold"
    print("The maximum temperature in New York for next week are", max_temp['temperatureMax'], "Fahrenheit or", round(Convert_Celsius), "Celsius. That is", Temperature)

The maximum temperature in New York for next week are 77.53 Fahrenheit or 25 Celsius. That is warm
The maximum temperature in New York for next week are 81.4 Fahrenheit or 27 Celsius. That is warm
The maximum temperature in New York for next week are 77.76 Fahrenheit or 25 Celsius. That is warm
The maximum temperature in New York for next week are 73.44 Fahrenheit or 23 Celsius. That is warm
The maximum temperature in New York for next week are 75.83 Fahrenheit or 24 Celsius. That is warm
The maximum temperature in New York for next week are 75.01 Fahrenheit or 24 Celsius. That is warm
The maximum temperature in New York for next week are 72.62 Fahrenheit or 23 Celsius. That is warm
The maximum temperature in New York for next week are 74.5 Fahrenheit or 24 Celsius. That is warm


In [62]:
 import datetime
print(datetime.datetime.fromtimestamp(int("1284101485")).strftime('%Y-%m-%d'))

2010-09-10


In [71]:
import time
import datetime
Forecast = weather_data['daily']['data']
for max_temp in Forecast:
    Convert_Celsius = (max_temp['temperatureMax'] - 32) * 5 / 9
    time1 = max_temp['time']
    if Convert_Celsius > 30:
        Temperature = "hot"
    elif Convert_Celsius > 20:
        Temperature = "comfortably warm."
    else:
        Temperature = "cold"
    #print("On", time.strftime('%A, %b %d', time.localtime(time1)), "the maximum temperature will be", max_temp['temperatureMax'], "Fahrenheit or", round(Convert_Celsius), "Celsius. That is", Temperature)
    print("On", time.strftime('%A', time.localtime(time1)), "the maximum temperature will be", max_temp['temperatureMax'], "Fahrenheit or", round(Convert_Celsius), "Celsius. That is", Temperature)

On Friday the maximum temperature will be 77.53 Fahrenheit or 25 Celsius. That is comfortably warm.
On Saturday the maximum temperature will be 81.4 Fahrenheit or 27 Celsius. That is comfortably warm.
On Sunday the maximum temperature will be 77.76 Fahrenheit or 25 Celsius. That is comfortably warm.
On Monday the maximum temperature will be 73.44 Fahrenheit or 23 Celsius. That is comfortably warm.
On Tuesday the maximum temperature will be 75.83 Fahrenheit or 24 Celsius. That is comfortably warm.
On Wednesday the maximum temperature will be 75.01 Fahrenheit or 24 Celsius. That is comfortably warm.
On Thursday the maximum temperature will be 72.62 Fahrenheit or 23 Celsius. That is comfortably warm.
On Friday the maximum temperature will be 74.5 Fahrenheit or 24 Celsius. That is comfortably warm.


In [72]:
import datetime
print(datetime.datetime.fromtimestamp(int(time)).strftime('%Y-%m-%d'))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'module'

6) What's the weather looking like for the rest of today in Miami, Florida? I'd like to know the temperature for every hour, and if it's going to have cloud cover of more than 0.5 say "{temperature} and cloudy" instead of just the temperature.

In [73]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/25.7617,-80.1918")
Florida = response.json()
Florida.keys()

dict_keys(['longitude', 'currently', 'daily', 'hourly', 'minutely', 'latitude', 'flags', 'timezone', 'offset'])

In [74]:
Florida['hourly'].keys()

dict_keys(['summary', 'icon', 'data'])

In [75]:
#print(Florida['hourly']['data'])

In [76]:
import time
import datetime
Hourly = Florida['hourly']['data']
Hourly = Hourly
for weather in Hourly:
    time = weather['time']
    stop_time = datetime.datetime.fromtimestamp(int(time)).strftime('%H')
    if stop_time == '01':
        break
    if weather['cloudCover'] > 0.5:
        cloudy = "and cloudy"
    else:
        cloudy = "not cloudy"
    print(datetime.datetime.fromtimestamp(int(time)).strftime('%H:%M'), "{", weather['temperature'], "°F}",cloudy)

10:00 { 84.66 °F} and cloudy
11:00 { 86.01 °F} and cloudy
12:00 { 86.45 °F} and cloudy
13:00 { 86.75 °F} and cloudy
14:00 { 87.24 °F} and cloudy
15:00 { 87.18 °F} and cloudy
16:00 { 86.24 °F} and cloudy
17:00 { 85.1 °F} and cloudy
18:00 { 84.04 °F} and cloudy
19:00 { 83.27 °F} and cloudy
20:00 { 82.31 °F} and cloudy
21:00 { 81.45 °F} and cloudy
22:00 { 80.96 °F} and cloudy
23:00 { 80.66 °F} and cloudy
00:00 { 79.94 °F} and cloudy


7) What was the temperature in Central Park on Christmas Day, 1980? How about 1990? 2000?

In [77]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.781750,-73.966641,346593600")
weather_data = response.json()
weather_data.keys()

dict_keys(['longitude', 'currently', 'hourly', 'daily', 'latitude', 'flags', 'timezone', 'offset'])

In [78]:
#print(weather_data['daily']['data'][0])

In [79]:
for Christmas in weather_data['daily']['data']:
    Convert_Celsius = (Christmas['temperatureMax'] - 32) * 5 / 9
    print("The maximum temperature on Christmas Day 1980 in Centralpark was", 
          Christmas['temperatureMax'], "Fahrenheit, or", round(Convert_Celsius), "degrees Celsius.")

The maximum temperature on Christmas Day 1980 in Centralpark was 29.53 Fahrenheit, or -1 degrees Celsius.


In [80]:
response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.781750,-73.966641,662126400")
weather_data_1990 = response.json()
weather_data_1990.keys()

response = requests.get("https://api.forecast.io/forecast/e554f37a8164ce189acd210d00a452e0/40.781750,-73.966641,977745600")
weather_data_2000 = response.json()
weather_data_2000.keys()

dict_keys(['longitude', 'currently', 'hourly', 'daily', 'latitude', 'flags', 'timezone', 'offset'])

In [81]:
for Christmas in weather_data_1990['daily']['data']:
    Convert_Celsius = (Christmas['temperatureMax'] - 32) * 5 / 9
    print("The maximum temperature on Christmas Day 1990 in Centralpark was", 
          Christmas['temperatureMax'], "Fahrenheit, or", round(Convert_Celsius), "degrees Celsius.")

The maximum temperature on Christmas Day 1990 in Centralpark was 32.02 Fahrenheit, or 0 degrees Celsius.


In [82]:
for Christmas in weather_data_2000['daily']['data']:
    Convert_Celsius = (Christmas['temperatureMax'] - 32) * 5 / 9
    print("The maximum temperature on Christmas Day 1980 in Centralpark was", 
          Christmas['temperatureMax'], "Fahrenheit, or", round(Convert_Celsius), "degrees Celsius.")

The maximum temperature on Christmas Day 1980 in Centralpark was 27.02 Fahrenheit, or -3 degrees Celsius.
